# Merged data
Loads the bdm data calculated in the different servers and saves it in a single file to be loaded by the deep learning model. 

In [2]:
import numpy as np
import scipy.sparse as sp
import pickle
from pybdm import BDM
from algorithms import PerturbationExperiment, NodePerturbationExperiment

### Loading data
The final version can receive as parameter the filename of the data shelf 

In [5]:
filename = './data_structures/ddi_bdm_se3_drugs562_juan8'
with open(filename, 'rb') as f:
    ddi = pickle.load(f)
    for key in ddi.keys():
        globals()[key]=ddi[key]
        print(key,"Imported successfully")

nodebdm_ddi_list Imported successfully
edgebdm_ddi_list Imported successfully
vms Imported successfully
rss Imported successfully
total_time Imported successfully
jobs Imported successfully


In [6]:
data = {}
data['vms_ddi'] = vms
data['time_ddi'] = total_time
data['rss_ddi'] = rss
with open('data_structures/decagon.pickle', 'wb') as f:
    # Pickle the 'data' dictionary using the highest protocol available.
    pickle.dump(data, f, protocol=2)

In [2]:
data_decagon = shelve.open("./data_structures/decagon_CMM")
print("Data Structures:")
for key in data_decagon:
    globals()[key]=data_decagon[key]
    print(key,"Imported successfully")
data_decagon.close()
print("BDM feature vectors:")
data_bdm = shelve.open("./data_structures/bdm_CMM")
for key in data_bdm:
    globals()[key]=data_bdm[key]
    print(key,"Imported successfully")
data_bdm.close()

Data Structures:
drug2idx Imported successfully
dti_adj Imported successfully
dti_genes Imported successfully
ppi_degrees Imported successfully
ddi_adj_list Imported successfully
prot_feat Imported successfully
ddi_degrees_list Imported successfully
se_mono_name2idx Imported successfully
drug_feat Imported successfully
gene2idx Imported successfully
dti_drugs Imported successfully
ppi_adj Imported successfully
BDM feature vectors:
rss Imported successfully
vms Imported successfully
bdm_drugs_dti Imported successfully
bdm_ddi_list Imported successfully
bdm_ppi Imported successfully
total_time Imported successfully
bdm_genes_dti Imported successfully


In [3]:
nodebdm_drugs_dti = np.array(bdm_genes_dti).reshape(-1,1)
nodebdm_genes_dti = np.array(bdm_drugs_dti).reshape(-1,1)

In [4]:
prot_feat = np.concatenate((prot_feat.toarray(),np.array(bdm_ppi).reshape(-1,1),
                           nodebdm_genes_dti),axis=1)

In [5]:
prot_feat.shape

(16235, 8)

In [6]:
arr_list = np.hstack([np.array(i[0]).reshape(-1,1) for i in bdm_ddi_list ])

In [7]:
np.shape(arr_list)

(476, 3)

In [8]:
nodebdm_drugs_dti.shape

(476, 1)

In [9]:
drug_feat.shape

(476, 9411)

In [10]:
drug_feat = np.concatenate((drug_feat.todense(),arr_list,nodebdm_drugs_dti),axis=1)
print(drug_feat.shape)

(476, 9415)


Re-exporting decagon variables to avoid inconsistencies among python versions

In [12]:
new_feat = shelve.open('./data_structures/new_feat.dat','n',protocol=2)
new_feat['drug_feat'] = sp.csr_matrix(drug_feat)
new_feat['prot_feat'] = sp.csr_matrix(prot_feat)
# Dictionaries
new_feat['gene2idx'] = gene2idx
new_feat['drug2idx'] = drug2idx
new_feat['se_mono_name2idx'] = se_mono_name2idx
#new_feat['se_combo_name2idx'] = se_combo_name2idx
# DDI
new_feat['ddi_adj_list'] = ddi_adj_list
new_feat['ddi_degrees_list'] = ddi_degrees_list
# DTI
new_feat['dti_adj'] = dti_adj
# PPI
new_feat['ppi_adj'] = ppi_adj
new_feat['ppi_degrees'] = ppi_degrees
new_feat.close()